In [1]:
import pandas as pd
import psycopg2

In [2]:
pg_creds = {
    "database": "mydatabase", 
    "host": "localhost", 
    "port": 5432, 
    "user": "myuser", 
    "password": "mypassword"
}
conn = psycopg2.connect(**pg_creds)

In [3]:
def get_results_as_table(query, conn=conn):
    with conn.cursor() as cursor:
        cursor.execute(query)

        data = pd.DataFrame(cursor.fetchall(), columns=[c.name for c in cursor.description])

    return data

Detecting anomalies based on transaction amounts. All transactions where the amount was more than 3 standard deviations away from the mean are considered anomalies. 

In [4]:
anomalies_query = '''
    WITH stats AS (
    SELECT
        cc_num,
        AVG(amt) AS mean_amt,
        STDDEV(amt) AS stddev_amt
    FROM transactions
    GROUP BY cc_num
    )
    SELECT
    t.*
    FROM transactions t
    JOIN stats s ON t.cc_num = s.cc_num
    WHERE t.amt > s.mean_amt + 3 * s.stddev_amt
    ORDER BY (t.amt - s.mean_amt) / s.stddev_amt DESC;
'''

anomalies = get_results_as_table(anomalies_query, conn)
anomalies.head(10)

,trans_num,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,merch_longitude,merch_index_right,merch_zip_code,merch_po_name,merch_state,merch_population,merch_pop_sqmi,merch_sqmi,merch_shape_length,merch_shape_area
0,b27f3f5928197f57c8b6fabe96b044c4,05-12-2020 17:08,630425000000,"fraud_Douglas, Schneider and Turner",shopping_pos,3479.09,Jennifer,Black,F,2870 Bean Terrace Apt. 756,...,-79.772467,4509.0,15401.0,Uniontown,PA,30508.0,576.82,52.89,1.041840368,0.014425034
1,a31d7c02e9343ced41730675788a182f,07-07-2020 13:27,377027000000000,fraud_Hackett Group,travel,1679.07,Jackie,Davis,F,1898 Parker Fork Apt. 057,...,-90.602953,21575.0,63901.0,Poplar Bluff,MO,34436.0,110.96,310.34,2.328972647,0.081130299
2,6882d55a84d2f4754ef3b4f68f7010da,27-09-2020 19:23,6011350000000000,fraud_Lesch Ltd,shopping_pos,3700.85,Samuel,Johnston,M,16285 Jessica Lights,...,-87.338959,11087.0,35579.0,Oakman,AL,2890.0,21.79,132.62,1.902800798,0.033386401
3,dfc4099c3e5ed3a655024efaa159c13f,10-12-2020 13:02,6011690000000000,"fraud_Hahn, Douglas and Schowalter",travel,6854.49,Victoria,Fleming,F,2807 Parker Station Suite 080,...,-93.316767,17900.0,55006.0,Braham,MN,3861.0,50.64,76.24,1.178844897,0.022826536
4,b47678d98ad7b0069e5b65c306236623,01-12-2020 13:19,6011650000000000,fraud_Friesen Inc,shopping_pos,1920.86,Kathryn,Smith,F,19838 Tonya Prairie Apt. 947,...,-92.134381,22109.0,65584.0,Saint Robert,MO,10976.0,431.96,25.41,0.485663926,0.006734708
5,bac59a292b44b14bad38cbd5e88118c6,10-10-2020 16:35,3575790000000000,fraud_Pouros-Haag,shopping_pos,1315.83,Lindsay,Wilson,F,7618 Gonzales Mission,...,-94.800835,22306.0,66062.0,Olathe,KS,79437.0,1965.29,40.42,0.689958067,0.010860787
6,8512b990d8a8395a8487871da9d7cad8,06-12-2020 13:11,4302480000000000,fraud_Kassulke PLC,shopping_net,1566.58,David,Rodriguez,M,821 Solis Points,...,-85.390386,15938.0,49601.0,Cadillac,MI,22074.0,101.29,217.92,1.907600381,0.063593353
7,462e0600ada45a23bd67fcab06298616,22-12-2020 21:54,5359540000000000,fraud_Boyer-Haley,travel,14637.79,Michael,Francis,M,1833 Jeanette Stravenue,...,-111.832758,19785.0,59720.0,Cameron,MT,637.0,0.73,873.80,3.010263756,0.258236721
8,55e42aee5cc9625fa01ed1b7796c6471,01-09-2020 15:56,180047000000000,fraud_Breitenberg LLC,travel,7436.65,Jared,Camacho,M,4257 Perez Mall,...,-81.151082,13825.0,44266.0,Ravenna,OH,33317.0,280.33,118.85,1.266496213,0.033029275
9,4ef348c1c92f0cfb5dd64d3e90e691b6,03-11-2020 12:20,36078100000000,fraud_Ankunding-Carroll,travel,5936.80,Christopher,Horn,M,956 Sanchez Highway,...,-82.740922,7282.0,24273.0,Norton,VA,5324.0,164.07,32.45,0.944727071,0.008503560


Checking the velocity at which transactions are made from credit cards. If we observe "rapid-fire" transactions, we flag them as potential fraud. 

In [5]:
velocity_pattern_query = '''
WITH velocity_check AS (
   SELECT
       *,
       LAG(unix_time) OVER (PARTITION BY cc_num ORDER BY unix_time) AS prev_time,
       unix_time - LAG(unix_time) OVER (PARTITION BY cc_num ORDER BY unix_time) AS time_diff
   FROM transactions
)
SELECT * FROM velocity_check
WHERE time_diff < 300
ORDER BY time_diff;
'''

velocity_analysis = get_results_as_table(velocity_pattern_query)
velocity_analysis.head(10)

,trans_num,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,merch_zip_code,merch_po_name,merch_state,merch_population,merch_pop_sqmi,merch_sqmi,merch_shape_length,merch_shape_area,prev_time,time_diff
0,08935e641fae6db8644e91cedd6204c0,16-09-2020 03:31,342952000000000,"fraud_Boehm, Predovic and Reinger",misc_pos,871.31,Kayla,Jones,F,6033 Young Track Suite 804,...,12508.0,Beacon,NY,19853.0,994.14,19.97,0.468494708,0.005576063,1379302259,2
1,a7b0e7daa0740e90defdfb8016a2c5d7,29-06-2020 23:55,3581130000000000,fraud_Brown PLC,misc_net,793.07,Gregory,Wood,M,622 Robert Summit,...,44041.0,Geneva,OH,13366.0,221.84,60.25,0.837840462,0.016898505,1372550116,2
2,abcbd471620adf0ef8040309147e7c30,20-08-2020 23:19,372382000000000,fraud_Medhurst PLC,shopping_net,956.23,Brent,Terrell,M,502 Rice Plaza Apt. 979,...,6516.0,West Haven,CT,55273.0,5020.25,11.01,0.320996425,0.003064315,1377040794,3
3,98559d91668d1edb28a75e441918aecc,16-11-2020 02:00,378006000000000,"fraud_Lockman, West and Runte",grocery_pos,251.97,Christopher,Matthews,M,433 John Heights Apt. 849,...,22030.0,Fairfax,VA,60813.0,3159.12,19.25,0.461121852,0.005173434,1384567214,3
4,7ee4e9bb090a4c3cfb0c01d5c2920455,14-12-2020 22:49,6011490000000000,"fraud_Greenholt, O'Hara and Balistreri",shopping_net,991.22,Gary,Barnes,M,2970 Flores Brooks,...,71855.0,Ozan,AR,633.0,8.60,73.64,0.794868702,0.018579443,1387061385,3
5,9b479c97ed9803de963e05f90e4085b5,25-08-2020 23:11,4512830000000000000,fraud_Runte-Mohr,food_dining,108.56,Monica,Cohen,F,864 Reynolds Plains,...,15370.0,Waynesburg,PA,13831.0,102.29,135.22,2.032271916,0.036868994,1377472279,5
6,248d9d8170352963a942cefcc3997881,28-12-2020 23:56,4716560000000000,"fraud_Daugherty, Pouros and Beahan",shopping_pos,5.64,Lauren,Anderson,F,11014 Chad Lake Apt. 573,...,59410.0,Augusta,MT,861.0,0.68,1258.02,4.183822250,0.388744288,1388274979,6
7,385c56fc9e425ec870e92afe01d22498,13-10-2020 23:44,213187000000000,fraud_Parker-Kunde,personal_care,80.33,Craig,Franco,M,9242 Vanessa Ramp Apt. 525,...,62675.0,Petersburg,IL,6026.0,52.54,114.70,1.226798868,0.031339897,1381707871,8
8,119fab045d8d0677d0e8c58a03c15962,20-11-2020 22:18,3582400000000000,fraud_Medhurst PLC,shopping_net,994.02,Michele,Patrick,F,557 Lynn Garden Suite 907,...,83814.0,Coeur D Alene,ID,28219.0,97.94,288.13,2.673500526,0.089318242,1384985918,8
9,65c4a80dabf9b440e884713d7dd79c30,11-12-2020 03:52,4560400000000000000,"fraud_Bahringer, Schoen and Corkery",shopping_pos,770.65,Angela,Taylor,F,6343 Ramirez Skyway Apt. 518,...,95963.0,Orland,CA,16932.0,40.69,416.16,2.779896223,0.113223116,1386733917,8


We check the time at which transactions were made. Transactions made late in the night may potentially be fraudulent.

In [6]:
late_night_txns_query = '''
   WITH transaction_counts AS (
      SELECT
         COUNT(*) AS total_transactions,
         SUM(CASE
                  WHEN EXTRACT(HOUR FROM to_timestamp(unix_time)) = 23 OR
                     EXTRACT(HOUR FROM to_timestamp(unix_time)) BETWEEN 0 AND 5
                  THEN 1
                  ELSE 0
            END) AS late_night_transactions
      FROM transactions
   ),
   late_night_details AS (
      SELECT
         trans_num,
         trans_date_trans_time,
         first,
         last,
         cc_num,
         merchant,
         category,
         amt,
         to_timestamp(unix_time) AS transaction_time,
         EXTRACT(HOUR FROM to_timestamp(unix_time)) AS hour_of_day
      FROM transactions
      WHERE EXTRACT(HOUR FROM to_timestamp(unix_time)) = 23 OR
            EXTRACT(HOUR FROM to_timestamp(unix_time)) BETWEEN 0 AND 5
   )
   SELECT
      (SELECT late_night_transactions FROM transaction_counts) AS late_night_count,
      (SELECT total_transactions FROM transaction_counts) AS total_count,
      (SELECT ROUND(late_night_transactions * 100.0 / total_transactions, 2)
      FROM transaction_counts) AS late_night_percentage,
      trans_num,
      transaction_time,
      hour_of_day,
      first || ' ' || last AS customer_name,
      RIGHT(cc_num::text, 4) AS card_ending,
      merchant,
      category,
      amt
   FROM late_night_details
   ORDER BY transaction_time;
'''

late_night_txns = get_results_as_table(late_night_txns_query)
late_night_txns.head(10)

,late_night_count,total_count,late_night_percentage,trans_num,transaction_time,hour_of_day,customer_name,card_ending,merchant,category,amt
0,13155,50000,26.31,f6838c01f5d2262006e6b71d33ba7c6d,2013-06-21 23:02:16+00:00,23,Brooke Smith,0000,fraud_Hermann and Sons,shopping_pos,842.65
1,13155,50000,26.31,d8103c37a1dbb77b12f083a597476478,2013-06-21 23:09:42+00:00,23,Jeffrey Smith,0000,fraud_Tromp Group,travel,569.96
2,13155,50000,26.31,7fcb13e83840ab985330ee52047fc949,2013-06-21 23:10:08+00:00,23,Peter Caldwell,0000,fraud_Abernathy and Sons,food_dining,60.70
3,13155,50000,26.31,0570c69c0045b82fb64171cf06c88110,2013-06-21 23:10:12+00:00,23,David Serrano,0000,fraud_Kris-Padberg,shopping_pos,125.84
4,13155,50000,26.31,1aaade4e34cdb73f009129aac8078bb4,2013-06-21 23:14:06+00:00,23,Brooke Rodriguez,0000,fraud_Turner and Sons,shopping_pos,8.82
5,13155,50000,26.31,0bac42920027f9b7b3d328bda2709983,2013-06-21 23:16:12+00:00,23,Ashley Cabrera,0000,fraud_Thompson-Gleason,health_fitness,22.55
6,13155,50000,26.31,9af0e55204bd89c3cdf909374dfaf464,2013-06-21 23:18:20+00:00,23,James Leon,0000,"fraud_Cole, Hills and Jewess",home,56.96
7,13155,50000,26.31,eb388ceabaf56474808664772b2d5853,2013-06-21 23:21:44+00:00,23,William Perry,0000,fraud_Kuhic LLC,shopping_net,1128.26
8,13155,50000,26.31,42744a022d53522b09676ef7f179d8d5,2013-06-21 23:23:20+00:00,23,Christine Davis,0000,"fraud_Lowe, Dietrich and Erdman",kids_pets,88.33
9,13155,50000,26.31,c5ac49af70a7eca02f2b23636b1d656c,2013-06-21 23:23:55+00:00,23,Nancy Roach,0000,fraud_Zulauf LLC,personal_care,2.63


Transactions that are made between locations that are more than a certain distance apart may seem suspicious. For our analysis, we set this threshold as 75 miles. 

In [7]:
geo_anomalies_query = '''
SELECT
 t.trans_num,
 t.trans_date_trans_time,
 t.first || ' ' || t.last AS customer_name,
 SUBSTRING(CAST(t.cc_num AS VARCHAR), LENGTH(CAST(t.cc_num AS VARCHAR)) - 3) AS card_ending,
 t.merchant AS merchant_name,
 t.amt,
 t.merch_is_fraud,
 (3959 * ACOS(
      COS(RADIANS(t.lat)) * COS(RADIANS(t.merch_lat)) *
      COS(RADIANS(t.merch_long) - RADIANS(t.long)) +
      SIN(RADIANS(t.lat)) * SIN(RADIANS(t.merch_lat))
 )) AS distance_miles
FROM transactions t
WHERE (3959 * ACOS(
      COS(RADIANS(t.lat)) * COS(RADIANS(t.merch_lat)) *
      COS(RADIANS(t.merch_long) - RADIANS(t.long)) +
      SIN(RADIANS(t.lat)) * SIN(RADIANS(t.merch_lat))
 )) > 75;
'''

geo_anomalies = get_results_as_table(geo_anomalies_query)
geo_anomalies

,trans_num,trans_date_trans_time,customer_name,card_ending,merchant_name,amt,merch_is_fraud,distance_miles
0,bcfb58b7c495d04190437fe7bcf647d2,25-12-2020 03:11,Kelsey Richards,0000,fraud_Cummings LLC,77.98,0.0,82.264095
1,3fa07778e1d491f2404223ddf0c5c2f9,16-08-2020 13:54,Jennifer Spencer,0000,fraud_Greenholt Ltd,102.69,0.0,81.192027
2,c8bc1b535f8ac28f83fc733fc868b353,02-09-2020 23:52,David Miller,0000,fraud_White and Sons,32.11,0.0,75.342670
3,8fe35102617c75243b14fb1497d82c9c,14-12-2020 21:46,Becky Mckinney,0000,fraud_Schmidt and Sons,5.29,0.0,82.638504
4,8f27ef3bdd62d7a7f06cf6a9904092cf,17-08-2020 18:52,Peter Edwards,0000,"fraud_Walter, Hettinger and Kessler",53.90,0.0,79.474756
...,...,...,...,...,...,...,...,...
2366,f8bb78aab0dbf79a7ecb83cd9d495c4e,09-11-2020 18:02,Jessica Perez,0000,fraud_Rippin-VonRueden,27.82,0.0,76.134124
2367,0a1de122f4e0b03b59ee084d8257993d,13-10-2020 05:00,Tanner Carroll,0000,fraud_Jaskolski-Vandervort,110.48,0.0,75.628148
2368,083eb28b4be113e41444a64b436bc9b3,03-10-2020 06:00,Christopher Castaneda,0000,fraud_Doyle Ltd,84.43,0.0,76.764212
2369,2ea610532333f839f0ffa7d6e4627ab9,14-08-2020 06:46,Curtis Young,0000,fraud_Dooley-Thompson,2.84,0.0,75.319874


In [8]:
geo_anomaly_summary_query = '''
WITH geo_anomaly_transactions AS (
 SELECT
   t.trans_num,
   t.trans_date_trans_time,
   t.first || ' ' || t.last AS customer_name,
   SUBSTRING(CAST(t.cc_num AS VARCHAR), LENGTH(CAST(t.cc_num AS VARCHAR)) - 3) AS card_ending,
   t.merchant AS merchant_name,
   t.amt,
   t.merch_is_fraud,
   (3959 * ACOS(
        COS(RADIANS(t.lat)) * COS(RADIANS(t.merch_lat)) *
        COS(RADIANS(t.merch_long) - RADIANS(t.long)) +
        SIN(RADIANS(t.lat)) * SIN(RADIANS(t.merch_lat))
   )) AS distance_miles
 FROM transactions t
 WHERE (3959 * ACOS(
        COS(RADIANS(t.lat)) * COS(RADIANS(t.merch_lat)) *
        COS(RADIANS(t.merch_long) - RADIANS(t.long)) +
        SIN(RADIANS(t.lat)) * SIN(RADIANS(t.merch_lat))
   )) > 75
)
SELECT
 COUNT(*) AS total_anomaly_transactions,
 SUM(CASE WHEN merch_is_fraud = 1 THEN 1 ELSE 0 END) AS fraudulent_transactions,
 CAST(
   100.0 * SUM(CASE WHEN merch_is_fraud = 1 THEN 1 ELSE 0 END) / NULLIF(COUNT(*), 0)
   AS NUMERIC(10,2)
 ) AS fraud_percentage,
 CAST(AVG(distance_miles) AS NUMERIC(10,2)) AS avg_distance_miles,
 MIN(distance_miles) AS min_distance_miles,
 MAX(distance_miles) AS max_distance_miles
FROM geo_anomaly_transactions;
'''


geo_anomalies_summary = get_results_as_table(geo_anomaly_summary_query)
geo_anomalies_summary

,total_anomaly_transactions,fraudulent_transactions,fraud_percentage,avg_distance_miles,min_distance_miles,max_distance_miles
0,2371,95,4.01,79.33,75.006834,91.346192


We also check whether transactions were made from the same credit card to merchants that are in different geographical locations, within a very short span of time. 

In [9]:
txn_time_consistency_query = '''
WITH rapid_transactions AS (
 WITH transaction_times AS (
   SELECT
      CONCAT(t.first, t.last, t.cc_num) AS customer_id,
      t.trans_num,
      t.trans_date_trans_time,
      t.city AS merchant_city,
      t.state AS merchant_state,
      t.merch_is_fraud,
      LEAD(t.trans_date_trans_time) OVER (PARTITION BY CONCAT(t.first, t.last, t.cc_num) ORDER BY t.trans_date_trans_time) AS next_trans_time,
      LEAD(t.city) OVER (PARTITION BY CONCAT(t.first, t.last, t.cc_num) ORDER BY t.trans_date_trans_time) AS next_city,
      LEAD(t.state) OVER (PARTITION BY CONCAT(t.first, t.last, t.cc_num) ORDER BY t.trans_date_trans_time) AS next_state,
      LEAD(t.merch_is_fraud) OVER (PARTITION BY CONCAT(t.first, t.last, t.cc_num) ORDER BY t.trans_date_trans_time) AS next_merch_is_fraud
   FROM transactions t
 )
 SELECT
   customer_id,
   trans_num,
   trans_date_trans_time,
   merchant_city,
   merchant_state,
   merch_is_fraud,
   next_trans_time,
   next_city,
   next_state,
   next_merch_is_fraud,
   EXTRACT(EPOCH FROM
     (TO_TIMESTAMP(next_trans_time, 'DD-MM-YYYY HH24:MI') -
      TO_TIMESTAMP(trans_date_trans_time, 'DD-MM-YYYY HH24:MI'))
   ) AS time_between_transactions
 FROM transaction_times
 WHERE next_trans_time IS NOT NULL
   AND EXTRACT(EPOCH FROM
     (TO_TIMESTAMP(next_trans_time, 'DD-MM-YYYY HH24:MI') -
      TO_TIMESTAMP(trans_date_trans_time, 'DD-MM-YYYY HH24:MI'))
   ) < 600  -- Less than 10 minutes
)
SELECT
 COUNT(*) AS total_rapid_transactions,
 SUM(CASE WHEN merch_is_fraud = 1 THEN 1 ELSE 0 END) AS fraudulent_transactions,
 CAST(
   100.0 * SUM(CASE WHEN merch_is_fraud = 1 THEN 1 ELSE 0 END) / NULLIF(COUNT(*), 0)
   AS NUMERIC(10,2)
 ) AS fraud_percentage,
 CAST(AVG(time_between_transactions) AS NUMERIC(10,2)) AS avg_seconds_between_trans,
 MIN(time_between_transactions) AS min_seconds_between_trans,
 MAX(time_between_transactions) AS max_seconds_between_trans
FROM rapid_transactions;
'''


txn_time_consistency = get_results_as_table(txn_time_consistency_query)
txn_time_consistency

,total_rapid_transactions,fraudulent_transactions,fraud_percentage,avg_seconds_between_trans,min_seconds_between_trans,max_seconds_between_trans
0,14438,320,2.22,-8453474.57,-15810180.000000,540.000000
